### Analyzing perfiles database

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time
import re

from Utils import TransantiagoConstants

In [3]:
import DailyPerfilesBuilder

#### Running for only last week

In [4]:
summary_dir = TransantiagoConstants.SummaryDir

In [5]:
dates = TransantiagoConstants.updateCurrentSSHDates()

In [6]:
start_date = '2017-08-21'

In [7]:
start_index = dates.index(start_date)

In [8]:
end_date = '2017-08-25'

In [9]:
end_index = dates.index(end_date)

In [10]:
dates = dates[start_index:end_index+1]

* Creating the dictionaries

In [11]:
dict_un_1 = {}
dict_un_3 = {}
dict_un_4 = {}
dict_un_5 = {}
dict_un_6 = {}
dict_un_7 = {}

* Number of services per UN to select

In [12]:
number_services = 20

In [13]:
i = 0
for date in dates:
    i = i+1
    print('This is the ' + str(i) + ' iteration')
    #Creating empty lists for selected services
    list_un_1 = []
    list_un_3 = []
    list_un_4 = []
    list_un_5 = []
    list_un_6 = []
    list_un_7 = []    
    perfiles_builder = DailyPerfilesBuilder.RunSilentlyDailyPerfilesBuilderClass(date) #Creating the object
#    perfiles_builder.runSimplifyPerfiles() #Simplifying perfiles
    perfiles_builder.loadSimplifiedPerfiles() #Loading perfiles into memory
    perfiles_builder.dropParaderos() #Droping paraderos info
    perfiles_builder.mergeTurnstileDatabase() #Merging turnstile info: critical
    perfiles_builder.groupByTurnstilePresence() #Grouping by servicio_sentido
    perfiles_builder.appendUnidadNegocio() #Appending 'unidad de negocio' column
    grouped_perfiles = perfiles_builder.grouped_data #Getting the info
    ### Filling the lists ###
    #1. Get ratio for all services in grouped_perfiles.
    grouped_perfiles['ratio_ct_st'] = grouped_perfiles['torniquete_mariposa_total']/grouped_perfiles['no_torniquete_total']
    grouped_perfiles['1_vs_ratio'] = abs(1-grouped_perfiles['ratio_ct_st'])
    #2. Replacing 'inf' by NaNs
    grouped_perfiles.replace([np.inf, -np.inf], np.nan, inplace=True)
    #3. Sorting by 'UN' and '1_vs_ratio' in ascending order and putting NaNs in last positions (by-default option)
    grouped_perfiles.sort_values(by=['UN','1_vs_ratio'],ascending=[True, True], inplace=True)
    #4. Select BY UN a pre-defined number of services.
    important_services = grouped_perfiles.groupby('UN').head(number_services).reset_index(drop=True)
#    file_name = 'important_services_' + date + '.csv'
#    summary_of_exps = os.path.join(summary_dir, file_name)
#    important_services.to_csv(summary_of_exps, sep=';', encoding='latin-1')
    list_un_1 = important_services.loc[important_services['UN']==1,'ServicioSentido'].tolist()
    list_un_3 = important_services.loc[important_services['UN']==3,'ServicioSentido'].tolist()
    list_un_4 = important_services.loc[important_services['UN']==4,'ServicioSentido'].tolist()
    list_un_5 = important_services.loc[important_services['UN']==5,'ServicioSentido'].tolist()
    list_un_6 = important_services.loc[important_services['UN']==6,'ServicioSentido'].tolist()
    list_un_7 = important_services.loc[important_services['UN']==7,'ServicioSentido'].tolist()    
    #5. Filling the dictionaries
    dict_un_1[date] = list_un_1
    dict_un_3[date] = list_un_3
    dict_un_4[date] = list_un_4
    dict_un_5[date] = list_un_5
    dict_un_6[date] = list_un_6
    dict_un_7[date] = list_un_7
    #Done...

This is the 1 iteration
Not found in BOTH turnstile databases: 36654
This is the 2 iteration
Not found in BOTH turnstile databases: 36997
This is the 3 iteration
Not found in BOTH turnstile databases: 36938
This is the 4 iteration
Not found in BOTH turnstile databases: 36858
This is the 5 iteration
Not found in BOTH turnstile databases: 36069


In [14]:
index = 0
first_date = dates[0]
for date in dates:
    if index==0:
        services_un_1 = set(dict_un_1[first_date])
        services_un_3 = set(dict_un_3[first_date])
        services_un_4 = set(dict_un_4[first_date])
        services_un_5 = set(dict_un_5[first_date])
        services_un_6 = set(dict_un_6[first_date])
        services_un_7 = set(dict_un_7[first_date])
    else:
        services_un_1 = services_un_1 & set(dict_un_1[date])
        services_un_3 = services_un_3 & set(dict_un_3[date])
        services_un_4 = services_un_4 & set(dict_un_4[date])
        services_un_5 = services_un_5 & set(dict_un_5[date])
        services_un_6 = services_un_6 & set(dict_un_6[date])
        services_un_7 = services_un_7 & set(dict_un_7[date])
    index=index+1

In [15]:
services_un_1

{'T106 00I',
 'T106 00R',
 'T108 00I',
 'T108 00R',
 'T120 00I',
 'T120 00R',
 'T130 E0 00I'}

In [16]:
services_un_3

{'T350 00I',
 'T350 00R',
 'T371 01R',
 'T379 00I',
 'T379 00R',
 'T379 C0 00I',
 'T379 C0 00R',
 'T381 00I',
 'T381 00R'}

In [17]:
services_un_4

{'T402 00I',
 'T402 00R',
 'T406 00I',
 'T406 00R',
 'T407 00I',
 'T407 00R',
 'T414 E0 00I',
 'T415 E0 00R',
 'T422 00I',
 'T422 00R',
 'T426 00I',
 'T426 00R',
 'T427 00I',
 'T427 00R',
 'T430 00I',
 'T430 00R'}

In [18]:
services_un_5

{'T523 00I', 'T535 C0 00R', 'T538 C0 00I', 'T538 C0 00R'}

In [19]:
services_un_6

{'B51 C0 00I', 'B55 00I', 'B56 00R'}

In [20]:
services_un_7

{'F52 00R'}

* Special treatment for UN2

In [21]:
date

'2017-08-25'

In [22]:
important_services_UN2 = grouped_perfiles.loc[grouped_perfiles['UN']==2,:].sort_values(by='torniquete_mariposa_total',ascending=False)

In [23]:
important_services_UN2.head(10)

,ServicioSentido,torniquete_mariposa_total,no_torniquete_total,TS_CODE,DIRECTION,UN,USER_CODE,ratio_ct_st,1_vs_ratio
321,T201 00R,98,0,201,Ret,2.0,201,NaN,NaN
320,T201 00I,93,0,201,Ida,2.0,201,NaN,NaN
330,T204 00I,69,0,204,Ida,2.0,204,NaN,NaN
331,T204 00R,60,0,204,Ret,2.0,204,NaN,NaN
327,T203 00R,55,0,203,Ret,2.0,203,NaN,NaN
348,T208 00I,54,0,208,Ida,2.0,208,NaN,NaN
322,T201 E0 00I,54,0,201,Ida,2.0,201,NaN,NaN
326,T203 00I,53,0,203,Ida,2.0,203,NaN,NaN
349,T208 00R,53,0,208,Ret,2.0,208,NaN,NaN
323,T201 E0 00R,51,0,201,Ret,2.0,201,NaN,NaN


In [24]:
important_services_UN2.loc[important_services_UN2['no_torniquete_total']!=0,:]

,ServicioSentido,torniquete_mariposa_total,no_torniquete_total,TS_CODE,DIRECTION,UN,USER_CODE,ratio_ct_st,1_vs_ratio


## Closed